In [1]:
import tweepy

def authenticate():
  BEARER_TOKEN = '###' # replace with your bearer token.
  # attempt authentication
  try:
    print("Authenticating...")
    # create Client object
    client = tweepy.Client(bearer_token=BEARER_TOKEN)
    print("API value:....   ", client)
    # return client object
    return client
  except Exception as e:
    print("Error: Authentication Failed", e)

In [2]:
API = authenticate()

Authenticating...
API value:....    <tweepy.client.Client object at 0x7fd191ef9340>


In [3]:
def get_tweets(query, start_time, end_time, next_token):

  # empty list to store parsed tweets
  tweets = []

  expansions = ['author_id,in_reply_to_user_id,geo.place_id']
  tweet_fields = ['id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source']
  user_fields =  ['id,name,username,created_at,description,public_metrics,verified']
  place_fields = ['full_name,id,country,country_code,geo,name,place_type']
  try:
    # call twitter api to fetch tweets
    fetched_tweets = API.search_all_tweets(query,
      end_time=end_time,
      start_time=start_time,
      expansions=expansions,
      tweet_fields=tweet_fields,
      place_fields=place_fields,
      user_fields=user_fields,
      next_token=next_token
    )
  
    return fetched_tweets

  except Exception as e:
    print("Error getting tweets", e)

In [4]:
result = get_tweets('elon musk', '2022-10-01T00:00:01Z', '2022-11-01T00:00:01Z',next_token=None)

print(result)

Response(data=[<Tweet id=1587232956740997120 text='RT @JedediahBila: Why are so many left wingers afraid of conversation? They can’t stand Elon Musk, Joe Rogan, anyone providing a forum for…'>, <Tweet id=1587232956388560897 text="RT @kitchen5203: This!\nWhy can't these right-wing people make an effort to be honest?\n\n#wtpBLUE  #DemVoice #DemCast \n\nMy Washington Post su…">, <Tweet id=1587232956157943811 text="RT @CultureWarsMag: More proof that it's an anti-Semitic canard to say that Jews control the media.\nhttps://t.co/bwa5Nx3IIO">, <Tweet id=1587232956053041152 text='Elon Musk finalmente compra @Twitter “por el futuro de la civilización”\nhttps://t.co/hflGacBbLr https://t.co/21QsysQ6lD'>, <Tweet id=1587232955591794695 text="Opinion | Elon Musk's hellscape. And ours. https://t.co/BUScKWNKRH via @msnbc">, <Tweet id=1587232955508076544 text='“This just in: Due to Elon Musk’s purchase of Twitter, VERY famous celebrities Carl Morton, Elizabeth Sanders, Gary Jenkins, Patricia Cronkendor

In [5]:
import pandas as pd
import csv
import time

def append_to_csv(result_set, file_name):

  # A counter variable
  counter = 0
  # Open OR create the target CSV file
  csv_file = open(file_name, "a", newline="", encoding='utf-8')
  csv_writer = csv.writer(csv_file)

  # Loop through each tweet
  for tweet in result_set.data:
    # We will create a variable for each item since some of the keys might not exist for some tweets
    # So we will account for that

    author_id = tweet['author_id']
    created_at = tweet['created_at']
    if ('geo' in tweet):   
        geo = tweet['geo']['place_id']
    else:
        geo = " "
    tweet_id = tweet['id']
    lang = tweet['lang']
    retweet_count = tweet['public_metrics']['retweet_count']
    reply_count = tweet['public_metrics']['reply_count']
    like_count = tweet['public_metrics']['like_count']
    quote_count = tweet['public_metrics']['quote_count']
    source = tweet['source']
    text = tweet['text']
        
    # Assemble all data in a list
    res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]        
    
    # Append the result to the CSV file
    csv_writer.writerow(res)
    counter += 1

  # When done, close the CSV file
  csv_file.close()

  # Print the number of tweets for this iteration
  print("# of Tweets added from this response: ", counter)

In [10]:
count = 0 # Counting tweets per time period
max_count = 100 # Max tweets per time period
flag = True
next_token = None
query = 'Parag Musk'
start_time = '2022-10-01T00:00:01Z'
end_time = '2022-11-01T00:00:01Z'
# Total number of tweets we collected from the loop
# Check if flag is true
def run_fetch(flag=True, next_token=None, count=0, total_tweets=0 ):
  if (flag is not True):
    return
  while flag:
    print("-------------------")
    print("Token: ", next_token)
    result = get_tweets(query=query, start_time=start_time, end_time=end_time, next_token=next_token) # get_tweet function called
    if hasattr(result, 'meta'):
      result_count = result.meta['result_count']
      if result_count is not None and result_count > 0:
        # print("Start Date: ", start_list[i])
        append_to_csv(result, "data.csv")
        count += result_count
        total_tweets += result_count
        print("Total # of Tweets added: ", total_tweets)
        print("-------------------")
        if result.meta['next_token']:
          # Save the token to use for next call
          next_token = result.meta['next_token']
          print("Next Token: ", next_token)
          time.sleep(5)
          run_fetch(True, next_token=next_token, count=count, total_tweets=total_tweets)
        else:
          # Since this is the final request, turn flag to false to move to the next time period.
          flag = False
          next_token = None
    time.sleep(5)
  print("Total number of results: ", total_tweets)

In [11]:
run_fetch()

-------------------
Token:  None
# of Tweets added from this response:  10
Total # of Tweets added:  10
-------------------
Next Token:  b26v89c19zqg8o3fpzemdy23r1oqtw3h5xjlczv97wtj1
-------------------
Token:  b26v89c19zqg8o3fpzemdy23r1oqtw3h5xjlczv97wtj1
# of Tweets added from this response:  10
Total # of Tweets added:  20
-------------------
Next Token:  b26v89c19zqg8o3fpzemdy23r1gurc1dhj22c1hxz3tz1
-------------------
Token:  b26v89c19zqg8o3fpzemdy23r1gurc1dhj22c1hxz3tz1
# of Tweets added from this response:  10
Total # of Tweets added:  30
-------------------
Next Token:  b26v89c19zqg8o3fpzemdy23r1942506c0qb07fycebnh
-------------------
Token:  b26v89c19zqg8o3fpzemdy23r1942506c0qb07fycebnh
# of Tweets added from this response:  9
Total # of Tweets added:  39
-------------------
Next Token:  b26v89c19zqg8o3fpzemdy23r11fiq0n5xc9s34p1g7st
-------------------
Token:  b26v89c19zqg8o3fpzemdy23r11fiq0n5xc9s34p1g7st
# of Tweets added from this response:  10
Total # of Tweets added:  49
-